<a href="https://polly.elucidata.io/manage/workspaces?action=open_polly_notebook&amp;source=github&amp;path=ElucidataInc%2Fpolly-python%2Fblob%2Fmain%2FAnalyse%2Fcohort_creation_demo.ipynb&amp;kernel=elucidata%2FGenomics-hail&amp;machine=medium" target="_parent"><img alt="Open in Polly" src="https://elucidatainc.github.io/PublicAssets/open_polly.svg"/></a>


# Welcome to Notebook demonstrating MVP release of cohort journey

The demo of this can be found in this video: https://www.loom.com/share/71c97f0cfd554a9fb2aa6256664c08a0

In [1]:
# please do not modify
from IPython.display import display_html
def restartkernel() :
    display_html("<script>Jupyter.notebook.kernel.restart()</script>",raw=True)

In [2]:
!sudo pip3 install polly-python --upgrade

Looking in indexes: https://pypi.org/simple, http://54.245.179.143:80/
     |████████████████████████████████| 52 kB 17.3 MB/s eta 0:00:01
     |████████████████████████████████| 50 kB 74.8 MB/s eta 0:00:01
     |████████████████████████████████| 1.5 MB 14.3 MB/s eta 0:00:01
     |████████████████████████████████| 306 kB 115.7 MB/s eta 0:00:01
     |████████████████████████████████| 149 kB 124.0 MB/s eta 0:00:01
     |████████████████████████████████| 96 kB 110.7 MB/s eta 0:00:01
Using legacy 'setup.py install' for Cerberus, since package 'wheel' is not installed.
Using legacy 'setup.py install' for retrying, since package 'wheel' is not installed.
  Attempting uninstall: certifi
    Found existing installation: certifi 2021.5.30
    Uninstalling certifi-2021.5.30:
      Successfully uninstalled certifi-2021.5.30
    Running setup.py install for retrying ... done
  Attempting uninstall: Cerberus
    Found existing installation: Cerberus 1.3.4
    Uninstalling Cerberus-1.3.4:
      Succ

In [ ]:
restartkernel() #Pause for a few seconds before the kernel is refreshed

In [1]:
import os
from polly.auth import Polly
from polly.omixatlas import OmixAtlas
from polly.cohort import Cohort
from polly.workspaces import Workspaces
import pandas as pd
import urllib.parse as urlparse
from urllib.parse import parse_qs
import numpy as np
import cmapPy

In [2]:
AUTH_TOKEN=(os.environ['POLLY_REFRESH_TOKEN']) # Obtain authentication tokens
Polly.auth(AUTH_TOKEN)
omixatlas = OmixAtlas()
cohort1 = Cohort()
cohort2 = Cohort()
workspaces = Workspaces()

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
pd.set_option('display.width', 500)

## Finding datasets to be added in the cohort

In [3]:
query = """SELECT src_dataset_id, disease, protein_position, amino_acids, sequencer, impact, variant_class, consequence, name
        FROM tcga.features AS features
        
        JOIN (
        SELECT dataset_id AS dataset_id, disease AS disease FROM tcga.datasets WHERE data_type LIKE 'Mutation'
         ) AS datasets
         ON features.src_dataset_id = datasets.dataset_id
         
         WHERE hugo_symbol IN ('TP53','PIK3CA','CDH1','GATA3') AND features.src_dataset_id LIKE '%BRCA%'
    ORDER BY features.src_dataset_id"""
results=omixatlas.query_metadata(query, query_api_version="v2")
results

Query execution succeeded (time taken: 58.37 seconds, data scanned: 72.752 MB)
Fetched 27 rows


,src_dataset_id,disease,protein_position,amino_acids,sequencer,impact,variant_class,consequence,name
0,BRCA_Mutation_TCGA-A1-A0SK-01A-12D-A099-09,['Breast Neoplasms'],133/393,M/K,Illumina Genome Analyzer II,MODERATE,SNV,missense_variant,TP53_c.398T>A
1,BRCA_Mutation_TCGA-A2-A3XT-01A-11D-A22X-09,['Breast Neoplasms'],135/393,C/R,Illumina HiSeq 2000;Illumina HiSeq 2500,MODERATE,SNV,missense_variant,TP53_c.403T>C
2,BRCA_Mutation_TCGA-A7-A0CE-01A-11W-A019-09,['Breast Neoplasms'],155/393,T/P,Illumina Genome Analyzer II,MODERATE,SNV,missense_variant,TP53_c.463A>C
3,BRCA_Mutation_TCGA-A8-A09Q-01A-11W-A019-09,['Breast Neoplasms'],755/1068,L,Illumina Genome Analyzer II,LOW,SNV,synonymous_variant,PIK3CA_c.2265A>G
4,BRCA_Mutation_TCGA-A8-A09Q-01A-11W-A019-09,['Breast Neoplasms'],224/393,NaN,Illumina Genome Analyzer II,HIGH,SNV,splice_donor_variant,TP53_c.672+2T>A
5,BRCA_Mutation_TCGA-AN-A0FL-01A-11W-A050-09,['Breast Neoplasms'],225/393,NaN,Illumina Genome Analyzer II,HIGH,SNV,splice_acceptor_variant,TP53_c.673-2A>G
6,BRCA_Mutation_TCGA-AN-A0FX-01A-11W-A050-09,['Breast Neoplasms'],248/393,R/W,Illumina Genome Analyzer II,MODERATE,SNV,missense_variant,TP53_c.742C>T
7,BRCA_Mutation_TCGA-AO-A03R-01A-21W-A050-09,['Breast Neoplasms'],265/393,L/R,Illumina HiSeq 2000,MODERATE,SNV,missense_variant,TP53_c.794T>G
8,BRCA_Mutation_TCGA-AR-A0U2-01A-11D-A10G-09,['Breast Neoplasms'],364/443,R/S,Illumina Genome Analyzer II,MODERATE,SNV,missense_variant,GATA3_c.1092A>T
9,BRCA_Mutation_TCGA-AR-A0U2-01A-11D-A10G-09,['Breast Neoplasms'],176/393,C/F,Illumina Genome Analyzer II,MODERATE,SNV,missense_variant,TP53_c.527G>T


### Creating list of datasets which are to be added in the cohort

In [4]:
moderate_data = results.loc[results['impact'] == 'MODERATE', 'src_dataset_id'].tolist()
high_data = results.loc[results['impact'] == 'HIGH', 'src_dataset_id'].tolist()

In [5]:
high_data

['BRCA_Mutation_TCGA-A8-A09Q-01A-11W-A019-09',
 'BRCA_Mutation_TCGA-AN-A0FL-01A-11W-A050-09',
 'BRCA_Mutation_TCGA-AR-A254-01A-21D-A167-09',
 'BRCA_Mutation_TCGA-D8-A1XO-01A-11D-A14K-09',
 'BRCA_Mutation_TCGA-EW-A1J2-01A-21D-A13L-09',
 'BRCA_Mutation_TCGA-LL-A50Y-01A-11D-A25Q-09']

In [6]:
moderate_data

['BRCA_Mutation_TCGA-A1-A0SK-01A-12D-A099-09',
 'BRCA_Mutation_TCGA-A2-A3XT-01A-11D-A22X-09',
 'BRCA_Mutation_TCGA-A7-A0CE-01A-11W-A019-09',
 'BRCA_Mutation_TCGA-AN-A0FX-01A-11W-A050-09',
 'BRCA_Mutation_TCGA-AO-A03R-01A-21W-A050-09',
 'BRCA_Mutation_TCGA-AR-A0U2-01A-11D-A10G-09',
 'BRCA_Mutation_TCGA-AR-A0U2-01A-11D-A10G-09',
 'BRCA_Mutation_TCGA-AR-A1AI-01A-11D-A12Q-09',
 'BRCA_Mutation_TCGA-AR-A255-01A-11D-A167-09',
 'BRCA_Mutation_TCGA-BH-A0HN-01A-11D-A099-09',
 'BRCA_Mutation_TCGA-BH-A1FU-01A-11D-A14G-09',
 'BRCA_Mutation_TCGA-C8-A27B-01A-11D-A167-09',
 'BRCA_Mutation_TCGA-D8-A145-01A-11D-A10Y-09',
 'BRCA_Mutation_TCGA-D8-A1XO-01A-11D-A14K-09',
 'BRCA_Mutation_TCGA-D8-A1XS-01A-11D-A14K-09',
 'BRCA_Mutation_TCGA-E2-A153-01A-12D-A12B-09',
 'BRCA_Mutation_TCGA-E9-A1NH-01A-11D-A14G-09',
 'BRCA_Mutation_TCGA-E9-A1RI-01A-11D-A167-09',
 'BRCA_Mutation_TCGA-EW-A1J2-01A-21D-A13L-09',
 'BRCA_Mutation_TCGA-LL-A50Y-01A-11D-A25Q-09']

### Creating cohort 1 (modetate impact) - while passing argument of repo and dataset

In this example, user has entered both optional parameters: repo_key and list of dataset_ids

#### create_cohort(local_path: str, cohort_name: str, description: str, repo_key= str (Optional), dataset_id= list (Optional)) 

In [7]:
cohort1.create_cohort("/import/tcga_cohort","moderate","tcga_tp53_brca_mutation_moderate","tcga",moderate_data)

INFO:root:'20' dataset/s added to Cohort!
INFO:root:Cohort Created !


### Creating cohort 2 (high impact) - without passing argument of repo and dataset

In this example, user has entered only the mandatory parameters: location, name and description of cohort

In [8]:
cohort2.create_cohort("/import/tcga_cohort","higher","tcga_tp53_brca_mutation_high")

INFO:root:Cohort Created !


### Addition of samples in cohort 2

In this example, user has entered both mandatory parameters: repo_key and list of dataset_ids

#### add_to_cohort(repo_key= str, dataset_id= list) 

In [9]:
cohort2.add_to_cohort("tcga",high_data)

INFO:root:'6' dataset/s added to Cohort!


### Summarizing the contents of the cohort

Pass two arguments for fetching the details of the cohort as shown below

In [10]:
metadata, cohort_details = cohort2.summarize_cohort()

In [11]:
metadata

{'cohort_name': 'higher',
 'number_of_samples': 6,
 'description': 'tcga_tp53_brca_mutation_high'}

In [12]:
cohort_details

,source_omixatlas,datatype,dataset_id
0,tcga,Mutation,BRCA_Mutation_TCGA-A8-A09Q-01A-11W-A019-09
1,tcga,Mutation,BRCA_Mutation_TCGA-AN-A0FL-01A-11W-A050-09
2,tcga,Mutation,BRCA_Mutation_TCGA-AR-A254-01A-21D-A167-09
3,tcga,Mutation,BRCA_Mutation_TCGA-D8-A1XO-01A-11D-A14K-09
4,tcga,Mutation,BRCA_Mutation_TCGA-EW-A1J2-01A-21D-A13L-09
5,tcga,Mutation,BRCA_Mutation_TCGA-LL-A50Y-01A-11D-A25Q-09


## Merging the sample level metadata for Cohort 2 (high.pco)

In [13]:
#cohort2.load_cohort("/import/cohort_tcga/higher.pco")
high_metadata = cohort2.merge_metadata()

/usr/local/lib/python3.8/site-packages/pandas/core/frame.py:4167: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [14]:
high_metadata

chd,BRCA_Pathology,BRCA_Subtype_PAM50,CNV.Clusters,DNA.Methylation.Clusters,HPV_Status,Included_in_previous_marker_papers,MSI_status,Mutation.Clusters,PARADIGM.Clusters,Pan.Gyn.Clusters,Protein.Clusters,Tumor.Type,Tumor_Grade,age_at_diagnosis,age_at_index,age_at_initial_pathologic_diagnosis,ajcc_pathologic_m,ajcc_pathologic_n,ajcc_pathologic_t,alcohol_history,barcode,bcr_patient_barcode,classification_of_tumor,curated_age_unit,curated_max_age,curated_min_age,days_to_birth.x,days_to_birth.y,days_to_collection,days_to_death.y,days_to_diagnosis,days_to_last_follow_up,days_to_last_followup,definition,demographic_id,diagnosis_id,disease_type,ethnicity,exposure_id,gender,icd_10_code,initial_weight,is_ffpe,kw_curated_cell_line,kw_curated_cell_type,kw_curated_disease,kw_curated_drug,kw_curated_genetic_mod_type,kw_curated_modified_gene,kw_curated_tissue,last_known_disease_status,lncRNA.Clusters,mRNA.Clusters,miRNA.Clusters,morphology,name,oct_embedded,paper_BRCA_Pathology,paper_BRCA_Subtype_PAM50,paper_CNV.Clusters,paper_DNA.Methylation.Clusters,paper_HPV_Status,paper_Included_in_previous_marker_papers,paper_MSI_status,paper_Mutation.Clusters,paper_PARADIGM.Clusters,paper_Pan.Gyn.Clusters,paper_Protein.Clusters,paper_Tumor.Type,paper_Tumor_Grade,paper_age_at_initial_pathologic_diagnosis,paper_days_to_birth,paper_days_to_death,paper_days_to_last_followup,paper_lncRNA.Clusters,paper_mRNA.Clusters,paper_miRNA.Clusters,paper_pathologic_stage,paper_patient,paper_tobacco_smoking_history,paper_vital_status,pathologic_stage,pathology_report_uuid,patient,primary_diagnosis,primary_site,prior_malignancy,prior_treatment,progression_or_recurrence,project_id,race,releasable,released,sample,sample_id,sample_submitter_id,sample_type,sample_type_id,shortLetterCode,site_of_resection_or_biopsy,state,submitter_id,subtype,synchronous_malignancy,tissue_or_organ_of_origin,tissue_type,tobacco_smoking_history,tumor_grade,tumor_stage,vital_status.x,vital_status.y,year_of_birth,year_of_diagnosis
cid,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
TCGA-A8-A09Q-01A-11W-A019-09,IDC,LumB,C4,C2,NaN,YES,NaN,C1,C4,NaN,NaN,BRCA,NaN,30528,83,83,M0,N2a,T4b,Not Reported,TCGA-A8-A09Q-01A-11R-A00Z-07,TCGA-A8-A09Q-01A,not reported,Years,83.0,83.0,-30528,-30528,637,NaN,0,761,761,Primary solid Tumor,201a11c2-b239-586b-b3e9-77bb581e4c20,e6815aee-36a7-501a-8644-826418c0f721,"c(""Ductal and Lobular Neoplasms""-- ""Complex Ep...",not reported,e6eaac2b-77c3-5f5d-b84b-229cfc92fefc,female,C50.9,100,FALSE,none,none,Breast Neoplasms,none,none,none,breast,not reported,NaN,C1,C3,8500/3,Breast Invasive Carcinoma,false,IDC,LumB,C4,C2,NaN,YES,NaN,C1,C4,NaN,NaN,BRCA,NaN,83,-30528,NaN,761,NaN,C1,C3,Stage_III,TCGA-A8-A09Q,NaN,Alive,Stage_III,46C0AB7B-82FC-4D06-9F26-22AEC8BEEDA7,TCGA-A8-A09Q,Infiltrating duct carcinoma-- NOS,Breast,no,No,not reported,TCGA-BRCA,not reported,TRUE,TRUE,TCGA-A8-A09Q-01A,6f8b48ba-ef24-4d33-a0c3-bc900a6c8cd2,TCGA-A8-A09Q-01A,Primary Tumor,1,TP,Breast-- NOS,released,TCGA-A8-A09Q,LumB,No,Breast-- NOS,Not Reported,NaN,not reported,stage iiib,Alive,Alive,1925,2008
TCGA-AN-A0FL-01A-11W-A050-09,Other,Basal,C4,C4,NaN,YES,NaN,C1,C2,C5,C5,BRCA,NaN,22805,62,62,M0,N0,T2,Not Reported,TCGA-AN-A0FL-01A-11R-A034-07,TCGA-AN-A0FL-01A,not reported,Years,62.0,62.0,-22805,-22805,69,NaN,0,231,231,Primary solid Tumor,1c532cbe-8112-5560-87fd-33a3362cb5ad,4f3eca7a-ba68-548d-9443-97205610c7b1,"c(""Ductal and Lobular Neoplasms""-- ""Complex Ep...",not hispanic or latino,c5cfd740-bf0e-5aad-b003-a9af5e81fa27,female,C50.9,310,FALSE,none,none,Breast Neoplasms,none,none,none,breast,not reported,C3,C4,C7,8500/3,Breast Invasive Carcinoma,false,Other,Basal,C4,C4,NaN,YES,NaN,C1,C2,C5,C5,BRCA,NaN,62,-22805,NaN,231,C3,C4,C7,Stage_II,TCGA-AN-A0FL,NaN,Alive,Stage_II,049A2CA6-951A-452F-963A-F54FE87ECA31,TCGA-AN-A0FL,Infiltrating duct carcinoma-- NOS,Breast,no,No,not reported,TCGA-BRCA,white,TRUE,TRUE,TCGA-AN-A0FL-01A,2f2d2e05-c6ea-4f29-8bed-04

### Merging the data-matrix for Cohort 2 (high.pco)

In [15]:
high_data_matrix = cohort2.merge_data_matrix()

In [16]:
high_data_matrix

cid,TCGA-A8-A09Q-01A-11W-A019-09,TCGA-AN-A0FL-01A-11W-A050-09,TCGA-AR-A254-01A-21D-A167-09,TCGA-D8-A1XO-01A-11D-A14K-09,TCGA-EW-A1J2-01A-21D-A13L-09,TCGA-LL-A50Y-01A-11D-A25Q-09
AARS2_c.1135G>C,1.0,NaN,NaN,NaN,NaN,NaN
ABCC6P1_n.1343A>G,1.0,NaN,NaN,NaN,NaN,NaN
ABI3BP_c.2581G>A,1.0,NaN,NaN,NaN,NaN,NaN
ADAMTS4_c.321G>A,1.0,NaN,NaN,NaN,NaN,NaN
ADCY8_c.2788C>A,NaN,1.0,NaN,NaN,NaN,NaN
AFF2_c.3343C>T,1.0,NaN,NaN,NaN,NaN,NaN
AFF3_c.1441C>A,NaN,1.0,NaN,NaN,NaN,NaN
AGFG2_c.985G>A,1.0,NaN,NaN,NaN,NaN,NaN
AMBRA1_c.3691G>C,NaN,1.0,NaN,NaN,NaN,NaN
AMY2B_c.-46-714C>A,NaN,NaN,NaN,NaN,NaN,1.0


### Remove samples from cohort

In [17]:
cohort2.remove_from_cohort(high_data[0:2])

INFO:root:'2' dataset/s removed from Cohort!


### Checking the validity of the cohort

In [18]:
cohort2.is_valid()

True

### Summarize cohort to verify if the samples were removed 

In [19]:
metadata, cohort_details = cohort2.summarize_cohort()

In [20]:
metadata

{'cohort_name': 'higher',
 'number_of_samples': 4,
 'description': 'tcga_tp53_brca_mutation_high'}

In [21]:
cohort_details

,source_omixatlas,datatype,dataset_id
0,tcga,Mutation,BRCA_Mutation_TCGA-AR-A254-01A-21D-A167-09
1,tcga,Mutation,BRCA_Mutation_TCGA-D8-A1XO-01A-11D-A14K-09
2,tcga,Mutation,BRCA_Mutation_TCGA-EW-A1J2-01A-21D-A13L-09
3,tcga,Mutation,BRCA_Mutation_TCGA-LL-A50Y-01A-11D-A25Q-09


## Editing the metadata of a cohort
Edit the name of the cohort and the description using this function

#### cohort.edit_cohort(name, description)

In [22]:
cohort2.edit_cohort("high_edited","tcga_tp53_brca_mutation_high_edited")

INFO:root:Cohort Name Updated!
INFO:root:Cohort Description Updated!


### Summarize confirm to confirm if the cohort has been edited 

In [23]:
metadata, cohort_details = cohort2.summarize_cohort()

In [24]:
metadata

{'cohort_name': 'high_edited',
 'number_of_samples': 4,
 'description': 'tcga_tp53_brca_mutation_high_edited'}

In [25]:
cohort_details

,source_omixatlas,datatype,dataset_id
0,tcga,Mutation,BRCA_Mutation_TCGA-AR-A254-01A-21D-A167-09
1,tcga,Mutation,BRCA_Mutation_TCGA-D8-A1XO-01A-11D-A14K-09
2,tcga,Mutation,BRCA_Mutation_TCGA-EW-A1J2-01A-21D-A13L-09
3,tcga,Mutation,BRCA_Mutation_TCGA-LL-A50Y-01A-11D-A25Q-09


## Delete the cohort

A cohort can be deleted by using delete_cohort() function

#### cohort.delete_cohort()

In [26]:
cohort1.delete_cohort()

INFO:root:Cohort Deleted Successfuly!


### Save cohort to the workspace

After editing the cohort, the user wants to save it in workspace for future use so they can use the following function

In [101]:
#Note: Preview of jpco and gct files are not available on worspace yet.

In [27]:
workspaces.upload_to_workspaces(workspace_id = 8861, workspace_path = "/high_edited.pco", 
                                local_path = "/import/tcga_cohort/high_edited.pco")

INFO:root:Upload successful on workspace-id=8861.


### Upload cohort from the workspace

In [28]:
workspaces.download_from_workspaces(workspace_id = 8861, workspace_path = "/high_edited.pco")

INFO:root:Download successful to path=/import/high_edited.pco


### Loading the cohort which was uploaded from the workspace

In [29]:
cohort1.load_cohort("/import/high_edited.pco")

INFO:root:Cohort Loaded !


### Checking valididty of the cohort which we just uploaded from workspace

In [30]:
cohort1.is_valid()

True

### Checking the summary of the cohort

In [31]:
metadata, cohort_details = cohort1.summarize_cohort()

In [32]:
metadata

{'cohort_name': 'high_edited',
 'number_of_samples': 4,
 'description': 'tcga_tp53_brca_mutation_high_edited'}

In [33]:
cohort_details

,source_omixatlas,datatype,dataset_id
0,tcga,Mutation,BRCA_Mutation_TCGA-AR-A254-01A-21D-A167-09
1,tcga,Mutation,BRCA_Mutation_TCGA-D8-A1XO-01A-11D-A14K-09
2,tcga,Mutation,BRCA_Mutation_TCGA-EW-A1J2-01A-21D-A13L-09
3,tcga,Mutation,BRCA_Mutation_TCGA-LL-A50Y-01A-11D-A25Q-09


### Adding files to cohort which was uploaded from workspace

In [34]:
cohort1.add_to_cohort("tcga",high_data[0:2])

INFO:root:'2' dataset/s added to Cohort!


### Verifying the files are added

In [35]:
metadata, cohort_details = cohort1.summarize_cohort()

In [36]:
metadata

{'cohort_name': 'high_edited',
 'number_of_samples': 6,
 'description': 'tcga_tp53_brca_mutation_high_edited'}

In [37]:
cohort_details

,source_omixatlas,datatype,dataset_id
0,tcga,Mutation,BRCA_Mutation_TCGA-AR-A254-01A-21D-A167-09
1,tcga,Mutation,BRCA_Mutation_TCGA-D8-A1XO-01A-11D-A14K-09
2,tcga,Mutation,BRCA_Mutation_TCGA-EW-A1J2-01A-21D-A13L-09
3,tcga,Mutation,BRCA_Mutation_TCGA-LL-A50Y-01A-11D-A25Q-09
4,tcga,Mutation,BRCA_Mutation_TCGA-A8-A09Q-01A-11W-A019-09
5,tcga,Mutation,BRCA_Mutation_TCGA-AN-A0FL-01A-11W-A050-09


### Please share feedback with @Yogesh or @Sabu on how we can improvise this journey further 